In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import os

import pandas as pd
from probeinterface.plotting import plot_probe

import random
import string
import sys
import shutil

import spikeinterface.full as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.curation as scur
import spikeinterface.widgets as sw
from spikeinterface.sortingcomponents.peak_detection import detect_peaks
from spikeinterface.sortingcomponents.peak_localization import localize_peaks

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from py_functions.spikeinterface_processing import load_recording_from_raw, load_probe_recording

In [ ]:
global_job_kwargs = dict(n_jobs=10, chunk_duration="1s")
si.set_global_job_kwargs(**global_job_kwargs)
plt.rcParams['figure.dpi'] = 250

In [ ]:
# RUN PARAMS
ROOT = '/data/Proyectos/Nanoneuro/data/rCxNs/'
SAMPLE_BASE = 'p2.+-1000mV.500us(000)'
well = (2, 3)
time_samplings_to_mask = [(1.07, 1.075), (3.07, 3.075), (5.07, 5.075), (7.07, 7.075)]
type_MEAS = 16  # 16 or 64

session_token = datetime.now().strftime("%y-%m-%d") + '_' + \
                ''.join(random.choice(string.ascii_letters) for i in range(8))

## Dataset loading & preprocesing

In [ ]:
recording = load_recording_from_raw(root=ROOT, sample_base=SAMPLE_BASE, well=well, time_samplings_to_mask=time_samplings_to_mask)
load_probe_recording(recording=recording, type_MEAS=type_MEAS)

In [ ]:
recording_bin = recording.save(n_jobs=8, chunk_duration="1s", folder=f'{ROOT}/tmp/bin_{session_token}')

recording_f = spre.bandpass_filter(recording_bin, freq_min=300, freq_max=5000)

recording_cmr = spre.common_reference(recording_f, reference='global', operator='median')

In [ ]:
recording_cmr

In [ ]:
si.plot_timeseries(recording_cmr)


## Peak detection

In [ ]:
# Noise detection - we use this beforehand to later use in peak detection

noise_levels = si.get_noise_levels(recording_cmr, return_scaled=False)

plt.hist(noise_levels)
plt.xlabel('noise  [uV]')

In [ ]:
local_radius = 150   # circle radius x 3

In [ ]:
peaks = detect_peaks(recording_cmr,
                     method='locally_exclusive',
                     local_radius_um=local_radius, 
                     detect_threshold=5,
                     noise_levels=noise_levels,
                    **global_job_kwargs)

peaks.shape

## Peak sorting

In [ ]:
recording_cmr

In [ ]:
sorting_TRDC2 = ss.run_sorter('tridesclous2', recording=recording_cmr, 
                              output_folder=f'{ROOT}/tmp/TRDC2_{session_token}', 
                              docker_image=False, 
                              apply_preprocessing=False)

print('Units found by tridesclous2:', sorting_TRDC2.get_unit_ids())

In [ ]:
sorting_MS4 = ss.run_sorter('mountainsort4', recording=recording_cmr, output_folder=f'{ROOT}/tmp/MS5_{session_token}', docker_image=False)
print('Units found by mountainsort4:', sorting_MS4.get_unit_ids())

In [ ]:
sorting_SPCR2 = ss.run_sorter('spykingcircus2', recording=recording_cmr, output_folder=f'{ROOT}/tmp/SPRC2_{session_token}', docker_image=False)
print('Units found by spykingcircus2:', sorting_SPCR2.get_unit_ids())

In [ ]:
sorting_pyKS = ss.run_sorter('pykilosort', recording=recording_cmr, output_folder=f'{ROOT}/tmp/pyKS_{session_token}', 
                             docker_image=True) 
print('Units found by pykilosort:', sorting_pyKS.get_unit_ids())

In [ ]:
we = si.extract_waveforms(recording_cmr, sorting_TRDC2, folder=f'{ROOT}/tmp/TRDC2_WF_{session_token}',load_if_exists=True,
    ms_before=1, ms_after=2., max_spikes_per_unit=500,
    n_jobs=1, chunk_size=30000)


In [ ]:
sw.plot_unit_waveforms(we)
